In [1]:
%cd ..

c:\Users\almas\Desktop\reviews_classification


In [2]:
import torch
import torch.nn as nn

from torchinfo import summary

from src.neuralnet.preprocess import prepare_data
from src.neuralnet.model import TabBERT
from src.neuralnet.train import train_model

In [3]:
TRANSFORMER_MODEL = 'prajjwal1/bert-tiny'  # 'bert-base-uncased'
MAX_LENGTH = 512
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
train_loader, val_loader, columns_idx, embedding_sizes, n_continuous = prepare_data(
    TRANSFORMER_MODEL, MAX_LENGTH, device, SEED
)

In [5]:
# Set seeds for random modules
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

# Initialize the model
model = TabBERT(embedding_sizes, n_continuous, TRANSFORMER_MODEL, device).to(device)

# Set up optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)  # Fine-tuning BERT with a small learning rate
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()  # Binary Cross-Entropy for binary classification

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
inputs, labels = next(iter(train_loader))

input_example = {
    'input_ids': inputs[:1][:, columns_idx['INPUT_IDS_START']:columns_idx['INPUT_IDS_END']],
    'attention_mask': inputs[:1][:, columns_idx['ATTENTION_MASK_START']:columns_idx['ATTENTION_MASK_END']],
    'x_cat': inputs[:1][:, columns_idx['CATEGORICAL_START']:columns_idx['CATEGORICAL_END']],
    'x_cont': inputs[:1][:, columns_idx['NUMERIC_START']:columns_idx['NUMERIC_END']]
}

pred_probab = model(**input_example)
print('probabilities:', pred_probab)

probabilities: tensor([[0.5092]], device='cuda:0', grad_fn=<SigmoidBackward0>)


In [9]:
summary(model, input_data=input_example)

Layer (type:depth-idx)                                            Output Shape              Param #
TabBERT                                                           [1, 1]                    --
├─BertForSequenceClassification: 1-1                              [1, 2]                    --
│    └─BertModel: 2-1                                             [1, 128]                  --
│    │    └─BertEmbeddings: 3-1                                   [1, 512, 128]             3,972,864
│    │    └─BertEncoder: 3-2                                      [1, 512, 128]             396,544
│    │    └─BertPooler: 3-3                                       [1, 128]                  16,512
│    └─Dropout: 2-2                                               [1, 128]                  --
│    └─Linear: 2-3                                                [1, 2]                    258
├─Softmax: 1-2                                                    [1, 2]                    --
├─ModuleList: 1-3           

In [ ]:
# input_names = ["Review Data"]
# output_names = ["Sentiment Prediction"]

# torch.onnx.export(model, input_example, "data/model.onnx", input_names=input_names, output_names=output_names)

In [ ]:
# Set seeds for random modules
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

# Train the model
train_model(model, train_loader, val_loader, columns_idx, criterion, optimizer, epochs=25)

Epoch [400/10000], Loss: 0.5483, Val Loss: 0.5374, Val ROC-AUC: 0.5255
Epoch [800/10000], Loss: 0.5440, Val Loss: 0.5343, Val ROC-AUC: 0.5390
Epoch [1200/10000], Loss: 0.5412, Val Loss: 0.5336, Val ROC-AUC: 0.5410
Epoch [1600/10000], Loss: 0.5422, Val Loss: 0.5331, Val ROC-AUC: 0.5403
Epoch [2000/10000], Loss: 0.5405, Val Loss: 0.5329, Val ROC-AUC: 0.5405
Epoch [2400/10000], Loss: 0.5418, Val Loss: 0.5326, Val ROC-AUC: 0.5414
Epoch [2800/10000], Loss: 0.5419, Val Loss: 0.5324, Val ROC-AUC: 0.5413
Epoch [3200/10000], Loss: 0.5403, Val Loss: 0.5322, Val ROC-AUC: 0.5414
Epoch [3600/10000], Loss: 0.5404, Val Loss: 0.5320, Val ROC-AUC: 0.5415
Epoch [4000/10000], Loss: 0.5389, Val Loss: 0.5318, Val ROC-AUC: 0.5416
Epoch [4400/10000], Loss: 0.5391, Val Loss: 0.5317, Val ROC-AUC: 0.5416


KeyboardInterrupt: 

In [ ]:
# adamw 1e-5
# Epoch [9600/10000], Loss: 0.5379, Val Loss: 0.5312, Val ROC-AUC: 0.5443
# Epoch [10000/10000], Loss: 0.5367, Val Loss: 0.5313, Val ROC-AUC: 0.5442